In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import random
from sklearn import datasets

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, make_scorer
from IPython.display import display
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from collections import Counter
#from imblearn.over_sampling import RandomOverSampler 

import os, ssl, sys

In [2]:
lfw_people = datasets.fetch_lfw_people(min_faces_per_person=50, resize=0.4)

face_data = lfw_people['data']
face_images = lfw_people['images']
face_tnames = lfw_people['target_names']
face_targets = lfw_people['target']

In [3]:
random_state = np.random.randint(0, 100000)
print('random_seed = ', random_state)
test_p = 0.3
val_p = 0.3  # Percentuale di dati di X_trainval da usare come validation set

random_seed =  43093


In [4]:
X_trainval, X_test, y_trainval, y_test, _, img_test = train_test_split(face_data, face_targets, face_images, test_size=test_p, random_state=random_state, shuffle=True)

display(pd.DataFrame({'X_trainval': X_trainval.shape, 'X_test': X_test.shape}, index=['N. sanmples', 'N.features']))

,X_trainval,X_test
N. sanmples,1092,468
N.features,1850,1850


In [5]:
# Preparazione PCA

n_comp_fda = X_trainval.shape[0] - face_tnames.shape[0]

pca = PCA(n_components= n_comp_fda)
pca.fit(X_trainval)

X_trainval_old = X_trainval.copy()
X_trainval = pca.transform(X_trainval)

X_test_old = X_test.copy()
X_test = pca.transform(X_test)

In [6]:
mda = LDA(solver='eigen', n_components=face_tnames.shape[0]-1)
mda.fit(X_trainval, y_trainval)


# Trasformazione dati. Salvare i vecchi in "copie di backup"

X_trainval_mda = mda.transform(X_trainval)
X_test_mda = mda.transform(X_test)

In [7]:
#oversampling
ros = RandomOverSampler(random_state= random_state)
y_trainval_old = y_trainval.copy()
X_trainval, y_trainval = ros.fit_resample(X_trainval, y_trainval)

NameError: name 'RandomOverSampler' is not defined

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Train')

y_train_ros_df = pd.DataFrame(y_trainval)

plt.bar(np.arange(len(face_tnames)), y_train_ros_df[0].value_counts(sort = False))
plt.xticks(np.arange(len(face_tnames)), labels = face_tnames, rotation = 65)
plt.grid()
plt.show()

In [ ]:
# Inizializzazione iper-parametri MLP
hidden_layer_sizes = [10]
activation = 'relu'
#300
patience = 300
#1000
max_epochs = 3000
verbose = False
batch_sz = 4

# Inizializzazione MLP
mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, batch_size=batch_sz, max_iter=max_epochs, early_stopping=True, n_iter_no_change=patience, random_state=random_state ,validation_fraction=val_p)

print(mlp.fit(X_trainval, y_trainval))

In [ ]:
y_pred_trainval = mlp.predict(X_trainval)
y_pred = mlp.predict(X_test)

acc_trainval = mlp.score(X_trainval, y_trainval)
prec_trainval = precision_score(y_trainval, y_pred_trainval, average='weighted')
rec_trainval = recall_score(y_trainval, y_pred_trainval, average='weighted')
f1_trainval = f1_score(y_trainval, y_pred_trainval, average='weighted')

acc = mlp.score(X_test, y_test)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

df_perf = pd.DataFrame({'Accuracy': [acc_trainval, acc], 
                        'Precision': [prec_trainval, prec], 
                        'Recall': [rec_trainval, rec],
                        'F1': [f1_trainval, f1]
                       },
                      index=['train. + val.', 'test'])

cmat = confusion_matrix(y_test, y_pred, labels=mlp.classes_)
cmat_norm_true = confusion_matrix(y_test, y_pred, labels=mlp.classes_, normalize='true')
cmat_norm_pred = confusion_matrix(y_test, y_pred, labels=mlp.classes_, normalize='pred')

df_cmat = pd.DataFrame(cmat, columns=face_tnames, index=face_tnames)
df_cmat_norm_true = pd.DataFrame(cmat_norm_true, columns=face_tnames, index=face_tnames)
df_cmat_norm_pred = pd.DataFrame(cmat_norm_pred, columns=face_tnames, index=face_tnames)

display(df_perf)
display(df_cmat)
display(df_cmat_norm_true)
display(df_cmat_norm_pred)